# **機器學習簡介**

在這個 Notebook 中，我們將介紹：
- 什麼是機器學習？
- 需要執行哪些步驟？
- 'Hello ML.NET World' - 訓練你的第一個 ML.NET 模型。

## **什麼是機器學習？**


```
var size = new HouseData() { Size = 2.5F };
var price = predictionEngine.Predict(大小);
```

上面的程式碼展示了如何使用已經訓練好的模型。 訓練模型的最終結果是一個函式，你可以將一些資料『HouseData.Size』傳遞給模型，它會給你一個預測——『Prediction.Price』。

上面是一個簡單的例子（可能太簡單了），但是模型可以接受更多的值。 例如 - [計程車資料集的價值預測/迴歸](https://ntbk.io/ml-e2e-taxi) --
是一個更復雜的例子，它接受`vendor_id`、`rate_code`、`passenger_count`、`trip_time_in_secs`、`trip_distance`、`payment_type`，然後預測`fare_amount`。

### **你是如何建立這個函式的？**
機器學習透過提供演算法資料並使用統計資料來預測值來建立或訓練該模型。 更多細節和範例如下！

## **需要執行哪些步驟？**

1. **獲取和準備訓練資料**
  為了訓練模型，我們需要標記資料。標記資料僅表示資料集中已經具有要預測的列的一堆資料，以便訓練演算法可以學習預測值。
1. **選擇訓練演算法並訓練**
    >**劇透**
    >在我們的大多數範例中，我們將使用 AutoML 來簡化此過程。 AutoML 策略性地為給定任務嘗試各種演算法和引數，以找出最適合您的資料的演算法和引數！
    >
    >你可以把它想象成一個花稍的 for 迴圈來嘗試所有的選項。我們的 AutoML 比這更聰明一些……但這基本上就是它的作用！
    >
    > 對於下面的範例，我們將訓練一個特定的演算法 - 這樣您就可以看到它是如何工作的！
    1. 選擇一個任務 - [ML.NET 任務](https://docs.microsoft.com/dotnet/machine-learning/resources/tasks)
    1. 選擇一種演算法 - [ML.NET 演算法](https://docs.microsoft.com/dotnet/machine-learning/how-to-choose-an-ml-net-algorithm)
    1. 設定演算法引數【詞彙表-超引數】(https://docs.microsoft.com/dotnet/machine-learning/resources/glossary#hyperparameter)
    1. 訓練——
        這是資料實際上被饋送到演算法以訓練模型的地方。這可能需要一些時間，具體取決於資料量、演算法和該演算法的引數。

1. **評估**
  一旦你訓練了一個模型——你怎麼知道它是有效的？有很多技術可以評估模型的效能。如果您想更深入地瞭解 - Checkout [Evaluation Metrics](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics)。否則，我們將在這些教程中提供範例。
1. **部署**
  在你訓練了一個模型之後......它只是 .NET 程式碼！建立它 交付它 - 但是您當前部署您的應用程式。

## 我該如何入門？
下面我們快速介紹一下 ML.NET——『Hello ML.NET World！』 該系列接下來的三個 Notebook 將分別深入探討 [資料準備和特徵工程](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/02-Data%20Preparation%20and%20Feature%20Engineering.ipynb)、[訓練和 AutoML](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/03-Training%20and%20AutoML.ipynb) 和 [模型評估](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/04-Model%20Evaluation.ipynb)

# **Hello ML.NET World!**
以下片段中的程式碼示範了最簡單的 ML.NET 應用程式。 此範例建立了一個線性迴歸模型，以使用房屋大小和價格資料來預測房價。

第一步是引用 [Microsoft.ML](https://www.nuget.org/packages/Microsoft.ML/) 套件。

關於這個 Notebook，我們在套件引用中新增引用如下：

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"

Installed Packages Microsoft.ML, 2.0.0-preview.22356.1

第二步是引用 ML.NET 命名空間：

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;

現在我們準備編寫程式碼來實現我們需要做的機器學習任務。 始終從建立 [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext?view=ml-dotnet) 開始，這是所有 ML.NET 操作的通用的做法

In [1]:
MLContext mlContext = new MLContext();

下一步是為我們要使用的資料定義資料結構。 這個樣本是關於房屋預測價格的。 開始定義以下包含房屋大小和價格的資料結構：

In [1]:
public class HouseData
{
    public float Size { get; set; }
    public float Price { get; set; }
}

然後定義房價預測資料結構

In [1]:
public class Prediction
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

現在我們準備好訓練我們將用於房價預測場景的預先收集的資料

In [1]:
HouseData[] houseData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.4F, Price = 3.7F } };

Using the `MLContext` we previously created, load the training data into ML.NET [IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) which is the fundamental ML.NET data type

In [1]:
IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

使用我們之前建立的『MLContext』，將訓練資料載入到 ML.NET [IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) 

In [1]:
// 2. Specify data preparation and model training pipeline
var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Size" })
               .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", maximumNumberOfIterations: 100));

建立估算器後，我們準備將通道中定義的轉換和訓練器應用於資料。 為此，請調用 [Fit](https://docs.microsoft.com/dotnet/api/microsoft.ml.iestimator-1.fit?view=ml-dotnet) 方法。

In [1]:
var model = pipeline.Fit(trainingData);

現在我們可以評估訓練好的模型。 方法是載入準備好的測試資料，然後呼叫 [Evaluate](https://docs.microsoft.com/dotnet/api/microsoft.ml.regressioncatalog.evaluate?view=ml-dotnet) 方法， 然後列印[確定係數](https://en.wikipedia.org/wiki/Coefficient_of_determination) 以瞭解如何使用測試資料擬合模型。 確定係數越接近 1，模型擬合效果越好。 重複訓練和評估步驟，直到從訓練的模型中獲得滿意的結果。

In [1]:
HouseData[] testData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.2F, Price = 1.5F },
    new HouseData() { Size = 1.4F, Price = 1.7F },
    new HouseData() { Size = 1.6F, Price = 1.9F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.2F, Price = 3.5F },
    new HouseData() { Size = 3.3F, Price = 3.6F },
    new HouseData() { Size = 3.5F, Price = 3.9F }, 
    new HouseData() { Size = 3.7F, Price = 4.3F }, 
    new HouseData() { Size = 4.0F, Price = 6.1F }, 
    new HouseData() { Size = 5.0F, Price = 7.2F }, 
    new HouseData() { Size = 6.0F, Price = 8.5F }, 
    new HouseData() { Size = 7.0F, Price = 9.8F }, 
    new HouseData() { Size = 8.0F, Price = 10.3F }, 
};

// Load the test data
IDataView trainingTestData = mlContext.Data.LoadFromEnumerable(testData);

// transform the test data using the model
IDataView transformedTestData = model.Transform(trainingTestData);

// Evaluate the model against the test data
RegressionMetrics trainedModelMetrics = mlContext.Regression.Evaluate(transformedTestData, labelColumnName: "Size");

// Print the R-Squared value. The Closer to 1 indicates a better fitted model.
Console.WriteLine($"Coefficient of determination for the trained model: {trainedModelMetrics.RSquared:0.00}");

Coefficient of determination for the trained model: 0.98


現在我們已經準備好訓練好的模型進行預測。 讓我們使用這個模型來預測樣本房價。 我們透過建立預測引擎 [PredictionEngine<TSrc,TDst>](https://docs.microsoft.com/dotnet/api/microsoft.ml.predictionengine-2?view=ml-dotnet) 來做到這一點。 預測引擎是用於對先前訓練的模型（和先前的轉換通道）進行單一預測的類。 從經過訓練的模型建立預測引擎可以透過以下程式碼完成：

In [1]:
var predictionEngine = mlContext.Model.CreatePredictionEngine<HouseData, Prediction>(model);

然後使用建立的預測引擎，我們可以預測房價如下：

In [1]:
var size = new HouseData() { Size = 2.5F };
var price = predictionEngine.Predict(size);
Console.WriteLine($"Predicted price for size: {size.Size*1000} sq ft= {price.Price*100:C}k");

Predicted price for size: 2500 sq ft= $274.48k


恭喜！ 您已使用自己的資料成功訓練了 ML.NET 迴歸模型，然後使用此模型預測房價。 這是一個總結建立和訓練 ML.NET 模型然後使用它來預測房價的端到端操作的圖表。

![](https://docs.microsoft.com/dotnet/machine-learning/media/mldotnet-annotated-workflow.png)

# **進階學習**

> [⏩ 下一個學習模組 - 資料準備和特徵工程](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/02-Data%20Preparation%20and%20Feature%20Engineering.ipynb)